---

# 🔍 Field and Model Validators

### 🎯 Intent

Add **custom validation logic** for individual fields or entire models using decorators.

---

### 🧩 Core Components

1. **📝 Field Validators**

   * Use `@field_validator("field_name")`.
   * Runs when validating that field.
   * Example use: minimum length, allowed characters, trimming.

2. **🏗 Model Validators**

   * Use `@model_validator(mode="before" | "after")`.
   * **before** → runs on raw input dict.
   * **after** → runs after fields validated (good for cross-field checks).

3. **⚡ Multiple Validators**

   * A field can have multiple validators.
   * Execution order: outermost → innermost.

4. **📋 Error Raising**

   * Raise `ValueError` or `TypeError`.
   * Pydantic wraps them into a structured `ValidationError`.

---

In [1]:
from pydantic import BaseModel, Field, ValidationError, field_validator, model_validator
import re

# 📝 Example model with both field and model validators
class UserSignup(BaseModel):
    username: str = Field(min_length=3, max_length=20)
    email: str
    password: str
    confirm_password: str

    # 1. 📝 Field validator → clean/validate username
    @field_validator("username")
    @classmethod
    def username_alphanumeric(cls, v: str) -> str:
        if not re.match(r"^[a-zA-Z0-9_]+$", v):
            raise ValueError("Username must be alphanumeric or underscore only")
        return v.lower().strip()  # normalize

    # 2. 📝 Field validator → enforce password length
    @field_validator("password")
    @classmethod
    def password_strength(cls, v: str) -> str:
        if len(v) < 6:
            raise ValueError("Password must be at least 6 characters")
        return v

    # 3. 🏗 Model validator → check password confirmation
    @model_validator(mode="after")
    def check_passwords(self):
        if self.password != self.confirm_password:
            raise ValueError("Passwords do not match")
        return self


# ✅ Example usage
try:
    user = UserSignup(
        username=" Mukesh_Y ",
        email="test@example.com",
        password="secret1",
        confirm_password="secret1"
    )
    print("Validated user:", user.model_dump())
except ValidationError as e:
    print("Errors:", e.errors())


# ❌ Example with errors
try:
    bad_user = UserSignup(
        username="@@bad@@",
        email="test@example.com",
        password="123",
        confirm_password="456"
    )
except ValidationError as e:
    from pprint import pprint
    pprint(e.errors())  # structured error info


Errors: [{'type': 'value_error', 'loc': ('username',), 'msg': 'Value error, Username must be alphanumeric or underscore only', 'input': ' Mukesh_Y ', 'ctx': {'error': ValueError('Username must be alphanumeric or underscore only')}, 'url': 'https://errors.pydantic.dev/2.11/v/value_error'}]
[{'ctx': {'error': ValueError('Username must be alphanumeric or underscore only')},
  'input': '@@bad@@',
  'loc': ('username',),
  'msg': 'Value error, Username must be alphanumeric or underscore only',
  'type': 'value_error',
  'url': 'https://errors.pydantic.dev/2.11/v/value_error'},
 {'ctx': {'error': ValueError('Password must be at least 6 characters')},
  'input': '123',
  'loc': ('password',),
  'msg': 'Value error, Password must be at least 6 characters',
  'type': 'value_error',
  'url': 'https://errors.pydantic.dev/2.11/v/value_error'}]
